In [59]:
import pandas as pd

In [60]:
df = pd.read_table('data/1.tsv')
df2 = pd.read_table('data/2.tsv')

In [61]:
df.pop('lensfiletime_utc')
df.head()

,time_kst,value
0,201605011300,1.24
1,201605011400,1.22
2,201605011500,1.20
3,201605011600,1.20
4,201605011700,1.20


In [62]:
df2.dropna(inplace=True)
df2.pop('lensfiletime_kst')
df2.head()

,targetdate12digit,class
0,2.016050e+11,0.0
1,2.016050e+11,0.0
2,2.016050e+11,1.0
3,2.016050e+11,1.0
4,2.016050e+11,1.0


In [63]:
df = df.join(df2.set_index('targetdate12digit'), on='time_kst')
df.head()

,time_kst,value,class
0,201605011300,1.24,0.0
1,201605011400,1.22,NaN
2,201605011500,1.20,NaN
3,201605011600,1.20,NaN
4,201605011700,1.20,NaN


In [64]:
# 시간 순으로 정렬
df.sort_values(by='time_kst', inplace=True)

In [65]:
# 결측값 채우기
df.fillna(0, inplace=True)
df.head()

,time_kst,value,class
0,201605011300,1.24,0.0
1,201605011400,1.22,0.0
2,201605011500,1.20,0.0
3,201605011600,1.20,0.0
4,201605011700,1.20,0.0


In [66]:
df['time'] = pd.to_datetime(df['time_kst'], format='%Y%m%d%H%M%S')
df.drop('time_kst', axis=1, inplace=True)

In [67]:
df.head()

,value,class,time
0,1.24,0.0,2016-05-01 13:00:00
1,1.22,0.0,2016-05-01 14:00:00
2,1.20,0.0,2016-05-01 15:00:00
3,1.20,0.0,2016-05-01 16:00:00
4,1.20,0.0,2016-05-01 17:00:00


In [68]:
df['month'] = df['time'].dt.month

In [69]:
x = df[['class', 'value', 'month']]
y = x.pop('class')
x.head()

,value,month
0,1.24,5
1,1.22,5
2,1.20,5
3,1.20,5
4,1.20,5


In [70]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=0, stratify=x['month'])

In [71]:
from xgboost import XGBClassifier

model = XGBClassifier()
model.fit(x_train, y_train)
score = model.score(x_test, y_test)
print(score)

0.9304075235109718


In [72]:
from flaml import AutoML
automl = AutoML()
automl.fit(x_train, y_train, task='classification')

[flaml.automl.automl: 02-13 09:27:11] {2716} INFO - task = classification
[flaml.automl.automl: 02-13 09:27:11] {2718} INFO - Data split method: stratified
[flaml.automl.automl: 02-13 09:27:11] {2721} INFO - Evaluation method: cv
[flaml.automl.automl: 02-13 09:27:11] {2848} INFO - Minimizing error metric: log_loss
[flaml.automl.automl: 02-13 09:27:11] {2898} WARNING - No search budget is provided via time_budget or max_iter. Training only one model per estimator. Zero-shot AutoML is used for certain tasks and estimators. To tune hyperparameters for each estimator, please provide budget either via time_budget or max_iter.
[flaml.automl.automl: 02-13 09:27:11] {2994} INFO - List of ML learners in AutoML Run: ['lgbm', 'rf', 'xgboost', 'extra_tree', 'xgb_limitdepth', 'lrl1']
[flaml.automl.automl: 02-13 09:27:11] {3323} INFO - iteration 0, current learner lgbm
[flaml.automl.automl: 02-13 09:29:30] {3461} INFO - Estimated sufficient time budget=10000s. Estimated necessary time budget=10s.
[f

/home/seokmin/workspace/azure_ml_test/.venv/lib/python3.10/site-packages/xgboost/sklearn.py:1395: UserWarning: `use_label_encoder` is deprecated in 1.7.0.
  warnings.warn("`use_label_encoder` is deprecated in 1.7.0.")
/home/seokmin/workspace/azure_ml_test/.venv/lib/python3.10/site-packages/xgboost/sklearn.py:1395: UserWarning: `use_label_encoder` is deprecated in 1.7.0.
  warnings.warn("`use_label_encoder` is deprecated in 1.7.0.")
/home/seokmin/workspace/azure_ml_test/.venv/lib/python3.10/site-packages/xgboost/sklearn.py:1395: UserWarning: `use_label_encoder` is deprecated in 1.7.0.
  warnings.warn("`use_label_encoder` is deprecated in 1.7.0.")
/home/seokmin/workspace/azure_ml_test/.venv/lib/python3.10/site-packages/xgboost/sklearn.py:1395: UserWarning: `use_label_encoder` is deprecated in 1.7.0.
  warnings.warn("`use_label_encoder` is deprecated in 1.7.0.")
/home/seokmin/workspace/azure_ml_test/.venv/lib/python3.10/site-packages/xgboost/sklearn.py:1395: UserWarning: `use_label_encode

[flaml.automl.automl: 02-13 09:31:30] {3508} INFO -  at 259.1s,	estimator xgboost's best error=0.2536,	best estimator xgboost's best error=0.2536
[flaml.automl.automl: 02-13 09:31:30] {3323} INFO - iteration 3, current learner extra_tree
[flaml.automl.automl: 02-13 09:32:41] {3508} INFO -  at 330.0s,	estimator extra_tree's best error=1.7283,	best estimator xgboost's best error=0.2536
[flaml.automl.automl: 02-13 09:32:41] {3323} INFO - iteration 4, current learner xgb_limitdepth


/home/seokmin/workspace/azure_ml_test/.venv/lib/python3.10/site-packages/xgboost/sklearn.py:1395: UserWarning: `use_label_encoder` is deprecated in 1.7.0.
  warnings.warn("`use_label_encoder` is deprecated in 1.7.0.")
/home/seokmin/workspace/azure_ml_test/.venv/lib/python3.10/site-packages/xgboost/sklearn.py:1395: UserWarning: `use_label_encoder` is deprecated in 1.7.0.
  warnings.warn("`use_label_encoder` is deprecated in 1.7.0.")
/home/seokmin/workspace/azure_ml_test/.venv/lib/python3.10/site-packages/xgboost/sklearn.py:1395: UserWarning: `use_label_encoder` is deprecated in 1.7.0.
  warnings.warn("`use_label_encoder` is deprecated in 1.7.0.")
/home/seokmin/workspace/azure_ml_test/.venv/lib/python3.10/site-packages/xgboost/sklearn.py:1395: UserWarning: `use_label_encoder` is deprecated in 1.7.0.
  warnings.warn("`use_label_encoder` is deprecated in 1.7.0.")
/home/seokmin/workspace/azure_ml_test/.venv/lib/python3.10/site-packages/xgboost/sklearn.py:1395: UserWarning: `use_label_encode

[flaml.automl.automl: 02-13 09:33:46] {3508} INFO -  at 395.4s,	estimator xgb_limitdepth's best error=0.2467,	best estimator xgb_limitdepth's best error=0.2467
[flaml.automl.automl: 02-13 09:33:46] {3323} INFO - iteration 5, current learner lrl1


/home/seokmin/workspace/azure_ml_test/.venv/lib/python3.10/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/home/seokmin/workspace/azure_ml_test/.venv/lib/python3.10/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/home/seokmin/workspace/azure_ml_test/.venv/lib/python3.10/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/home/seokmin/workspace/azure_ml_test/.venv/lib/python3.10/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


[flaml.automl.automl: 02-13 09:33:56] {3508} INFO -  at 405.1s,	estimator lrl1's best error=0.3362,	best estimator xgb_limitdepth's best error=0.2467


/home/seokmin/workspace/azure_ml_test/.venv/lib/python3.10/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/home/seokmin/workspace/azure_ml_test/.venv/lib/python3.10/site-packages/xgboost/sklearn.py:1395: UserWarning: `use_label_encoder` is deprecated in 1.7.0.
  warnings.warn("`use_label_encoder` is deprecated in 1.7.0.")


[flaml.automl.automl: 02-13 09:34:11] {3772} INFO - retrain xgb_limitdepth for 15.5s
[flaml.automl.automl: 02-13 09:34:11] {3779} INFO - retrained model: XGBClassifier(base_score=None, booster=None, callbacks=[],
              colsample_bylevel=0.6769490906308069, colsample_bynode=None,
              colsample_bytree=1.0, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, gpu_id=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=0.08466828646360688,
              max_bin=None, max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=7, max_leaves=None,
              min_child_weight=30.424259012001368, missing=nan,
              monotone_constraints=None, n_estimators=313, n_jobs=-1,
              num_parallel_tree=None, objective='multi:softprob',
              predictor=None, ...)
[flaml.automl.automl: 02-13 09:34:11

In [74]:
automl.score(x_test, y_test)

0.930282131661442